# First submission! Just submit this preliminary work to see if the scoreboard works ;)

More to come...

## Shopee Project - Determine if two products are the same by their images

The general workflow is:

### Image
* Stacking two backbone models for transfer learning: ResNet50 pretrained on MS-COCO (downloaded from the Internet) + ResNet50 pretrained on ImageNet (from Keras) => 
* Supervised feature selection by ANOVA => 
* Agglomerative clustering by cosine similarity

### Title
* Tokenize => Indonesian Stemmer + WordNet Lemmatizer => 
* Calculate TF-IDF =>
* Clustering

Merge the results from the image cluster and title cluster

In [ ]:
from shutil import copytree
#copytree('../input/zzxjoanw/hdbscan-0.8.27/hdbscan-0.8.27','./hdbscan')

In [ ]:
%cd hdbscan

In [ ]:
!python setup.py bdist_wheel

In [ ]:
%cd dist

In [ ]:
!pip install "hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl" --no-cache-dir --no-binary :all: --no-build-isolation
import hdbscan

In [ ]:
%cd /kaggle/working

In [ ]:
# from nltk.cluster.kmeans import KMeansClusterer
# from nltk.cluster.util import cosine_distance

In [ ]:
!pip install "../input/zzxjoanw/nltk-3.6.2-py3-none-any.whl"

In [ ]:
import numpy as np
import pandas as pd

import os

In [ ]:
import struct

# Image tools
#import imgaug

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation, Average, Concatenate, Conv2D, Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

import scipy
from sklearn import *

from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

# NLP tools
from html import unescape
from nltk.util import everygrams
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input as ppi_vgg16, decode_predictions as dp_vgg16
from keras.applications.resnet50 import ResNet50, preprocess_input as ppi_resnet50, decode_predictions as dp_resnet50
from keras.utils import plot_model

from joblib import load

# from glob import glob
# from shutil import copy2
from gc import collect

In [ ]:
import cv2
import numpy as np

def preprocess(path):
    #print(path)
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_RGB2BGR)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 205, 255, cv2.THRESH_BINARY_INV)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)

    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

    small1 = []
    small2 = []
    large1 = []
    large2 = []

    for cnt in cnts:
        x,y,w,h = cv2.boundingRect(cnt)
#         if w*h < 1000:
#             continue
        small1.append(y)
        small2.append(x)
        large1.append(y+h)
        large2.append(x+w)

    dst = img[np.min(small1):np.max(large1), np.min(small2):np.max(large2)]

    s = max(dst.shape[0:2])

    f = np.full((s,s,3),255,np.uint8)

    ax,ay = (s - dst.shape[1])//2,(s - dst.shape[0])//2

    f[ay:dst.shape[0]+ay,ax:ax+dst.shape[1]] = dst

    f = cv2.resize(f, (300, 300))
    
    return f

In [ ]:
# Indonesian stemmer

!pip install '/kaggle/input/zzxjoanw/PySastrawi-1.2.0-py2.py3-none-any.whl'
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [ ]:
df0 = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
#df0.sort_values('label_group',inplace=True)

In [ ]:
testpaths = df0['image'].tolist()
titles = df0['title'].tolist()
textid = df0['posting_id'].tolist()

In [ ]:
cntvect = feature_extraction.text.CountVectorizer(stop_words = 'english', max_features = 20000)
cntvect.fit(titles)
text_X = cntvect.transform(titles)
tf_trans = feature_extraction.text.TfidfTransformer(use_idf=True, norm='l2')
text_Xtf = tf_trans.fit_transform(text_X)
text_Xtf = text_Xtf.toarray()

In [ ]:
from numpy import *
mul_batch_size = 1024
myMatching = []
batch_num =shape(text_Xtf)[0]//mul_batch_size  
if shape(text_Xtf)[0]%mul_batch_size !=0: 
    batch_num += 1 

overall_ind = 0
print(batch_num)
for batch_index in range(0, batch_num): 
    print("batch:", batch_index)
    start = batch_index * mul_batch_size
    end = (batch_index+1) * mul_batch_size
    if end >= shape(text_Xtf)[0]:
        end = shape(text_Xtf)[0]
    cur_similarity = matmul(text_Xtf, text_Xtf[start:end].T).T
    for sample in cur_similarity:
            cur_matching = []
            for index,value in enumerate(sample):
                if value > 0.7:
                    cur_matching.append(textid[index])
            #cur_matching = ' '.join(cur_matching)
            myMatching.append(cur_matching)
            overall_ind += 1

In [ ]:
#lsa = decomposition.TruncatedSVD(n_components=250, random_state=0)
#lsa.fit(X_trainx[0:50000])
#X_trainx = lsa.transform(X_trainx)

In [ ]:
from tensorflow.python.keras.applications.efficientnet import EfficientNetB3

In [ ]:
# !wget https://dord.mynetgear.com/modelSelection13.joblib
#skb = load('/kaggle/input/zzxjoanw/goodman_1100.joblib')

In [ ]:
model_b4_imagenet = EfficientNetB3(weights='/kaggle/input/zzxjoanw/b3_goodman_.h5', include_top=False, input_shape=(300,300,3))
tt = model_b4_imagenet.get_layer('top_activation').output
avg_pool = GlobalAveragePooling2D(name='avg_pool')(tt)

model = Model(inputs=model_b4_imagenet.input,
          outputs=avg_pool)
# print(model_b4_imagenet.summary())
ins = np.zeros((0,1536))
batch = []

images = np.zeros(shape=(1, 300, 300, 3))
#img = preprocess(f'/kaggle/input/shopee-product-matching/train_images/{testpaths[0]}')
img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{testpaths[0]}') 

x_expand = np.expand_dims(img, axis=0)
images[0, :, :, :] = x_expand

batch.append(images)
for i, path in enumerate(testpaths[1:]):
#     start = time.time()
    images = np.zeros(shape=(1, 300, 300, 3))
    img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{path}') 
    #img = preprocess(f'/kaggle/input/shopee-product-matching/test_images/{testpaths[path]}') 
    
    x_expand = np.expand_dims(img, axis=0)
    images[0, :, :, :] = x_expand
    
    batch.append(images)
    if i % 64 == 0:
        batch = np.vstack(batch)
        images_features = model.predict(batch, batch_size=64)
        batch = []
        ins = np.vstack([ins,images_features])
if i % 64 != 0:
    batch = np.vstack(batch)
    images_features = model.predict(batch, batch_size=i % 64)
    batch = []
    ins = np.vstack([ins,images_features])

In [ ]:
#ins = skb.transform(ins)

In [ ]:
#ins = np.hstack([ins,X_trainx]) 

ss = preprocessing.StandardScaler()
ins = ss.fit_transform(ins)
print(shape(ins))

In [ ]:
pca = decomposition.PCA(n_components=min(800, shape(ins)[0]))
ins = pca.fit_transform(ins)

In [ ]:
modelAc = hdbscan.HDBSCAN(min_cluster_size=2, cluster_selection_epsilon=11,core_dist_n_jobs=-1)#,approx_min_span_tree=True, cluster_selection_method='leaf')
modelAclabels_ = modelAc.fit_predict(ins)

In [ ]:
#print(modelAclabels_)

In [ ]:
sub_result_dict = {}

#out = []
postingids = np.array(df0['posting_id'])

for i in range(modelAclabels_.shape[0]):
    outstring_list = []
    #print(modelAclabels_)
    if modelAc.labels_[i] == -1:
        outstring_list.append(postingids[i])
    else:
        outstring_list = postingids[modelAclabels_ == modelAclabels_[i]]          #predictions == predictions[i]])
    final_list = list(set(outstring_list).union(set(myMatching[i])))
    if(len(final_list) > 50):
        final_list = final_list[0:50]
    final_list = " ".join(final_list)
    #out.append(final_list)
    
    sub_result_dict[postingids[i]] = []
    sub_result_dict[postingids[i]].append(postingids[i])
    sub_result_dict[postingids[i]].append(final_list)
#print(out)
    

In [ ]:
def write_csv_kaggle_sub2(fname, result):
    # fname = file name
    # Y is a list/array with class entries
    
    # header
    df = pd.DataFrame.from_dict(result, columns = ['posting_id', 'matches'], orient='index')
    df.to_csv(path_or_buf = fname, index = False)

In [ ]:
#df0.insert(1,'matches',out)

In [ ]:
#df0[['posting_id','matches']].to_csv('submission.csv',index=False)

In [ ]:
write_csv_kaggle_sub2("/kaggle/working/submission.csv", sub_result_dict)